# What's the effective vocabulary size with inversions?

- min : 3  (1, b3, 5)
- maj : 3  (1, 3, 5)
- dim : 3  (1, b3, b5)
- aug : 3  (1, 3, #5)
- min7 : 4 (1, b3, 5, b7)
- maj7 : 4 (1, 3, 5, 7)
- dom7 : 4 (1, 3, 5, b7)
- dim7 : 4 (1, b3, b5, b7)
- hdim7 : 4 (1, b3, b5, 7)
- minmaj7 : 4 (1, b3, 5, 7)
- min6 : 4 (1, b3, 5, 6)
- maj6 : 4 (1, 3, 5, 6)
- sus2 : 3 (1, 2, 5)
- sus4 : 3 (1, 4, 5)
- X : 1
- N : 1

- Without inversions: $12 * (4 + 6 + 2 + 2) + 2 = 170$
- With inversions: $12 * (3 * 4 + 4 * 6 + 4 * 2 + 3 * 2) + 2 = 602$

In [1]:
import os
import jams
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from joblib import Parallel, delayed

In [2]:
def getid(fname):
    return os.path.splitext(os.path.basename(fname))[0]

In [3]:
def compare(ref_file, est_file, corpus):
    
    ref_jam = jams.load(ref_file, validate=False)
    est_jam = jams.load(est_file, validate=False)
    
    ref_ann = ref_jam.search(corpus=corpus)[0]
    est_ann = est_jam.annotations[0]
    est_ann = est_ann.trim(0, ref_ann.data[-1].time + ref_ann.data[-1].duration)
    
    results = jams.eval.chord(ref_ann, est_ann)
    
    return {getid(est_file): dict(results)}

In [4]:
def eval_model(ids, corpus, path):
    
    results = {}
    for tid in tqdm(ids):
        try:
            ref_file = os.path.join(REF_PATH, os.path.extsep.join([tid, 'jams']))
            est_file = os.path.join(path, os.path.extsep.join([tid, 'jams']))
            results.update(compare(ref_file, est_file, corpus))
        except ValueError as exc:
            print('Failure at track {}'.format(est_file))
            print(exc)
            
    return pd.DataFrame.from_dict(results, orient='index').sort_index(axis=1)

In [5]:
REF_PATH = '/home/bmcfee/data/eric_chords/references_v2/'

In [6]:
# MARL-mir analysis
marl_ids = pd.read_csv('/home/bmcfee/git/ace-lessons/data/chord_dataset/marl-chords.csv', squeeze=True, header=None)
marl_corp = 'MARL-Chords'

In [29]:
marl_cr2tvsi = eval_model(marl_ids, marl_corp, '/home/bmcfee/working/chords/model_deep_tv_struct_aug/predict_struct//')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [8]:
marl_cr2psi = eval_model(marl_ids, marl_corp, '/home/bmcfee/working/chords//model_deep_pre_struct_aug/predict_struct//')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [51]:
marl_cr2si = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions-inv/')

A Jupyter Widget

In [86]:
marl_cr2s = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions/')

A Jupyter Widget

/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [52]:
marl_cr2 = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_aug/predictions/')

A Jupyter Widget

In [53]:
marl_cr1si = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions-inv/')

A Jupyter Widget

In [57]:
marl_cr1s = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions/')

A Jupyter Widget

In [54]:
marl_cr1 = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_aug/predictions/')

A Jupyter Widget

In [7]:
marl_chordino = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

A Jupyter Widget

In [66]:
marl_dnn = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_dnn/predictions/')

A Jupyter Widget

In [92]:
marl_khmm = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_khmm/predictions/')

A Jupyter Widget

/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [16]:
marl_cr2npsi = eval_model(marl_ids, marl_corp, '/home/bmcfee/working/chords/model_noskip_pre_struct_aug/predict_struct/')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [23]:
marl_cr2nsi = eval_model(marl_ids, marl_corp, '/home/bmcfee/working/chords/model_noskip_struct_aug/predict_struct/')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [93]:
marl_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_chordino.json')
marl_dnn.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_dnn.json')
marl_khmm.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_khmm.json')

marl_cr1.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr1.json')
marl_cr1s.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr1s.json')
marl_cr1si.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr1si.json')

marl_cr2.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2.json')
marl_cr2s.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2s.json')
marl_cr2si.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2si.json')


In [30]:
marl_cr2tvsi.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2tvsi.json')

In [24]:
marl_cr2nsi.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2nsi.json')

In [17]:
marl_cr2npsi.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2npsi.json')

In [9]:
marl_cr2psi.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2psi.json')

In [10]:
# Isophonics
iso_ids = pd.read_csv('/home/bmcfee/git/ace-lessons/data/chord_dataset/isophonics.csv', squeeze=True, header=None)
iso_corp = 'Isophonics'

In [31]:
iso_cr2tvsi = eval_model(iso_ids, iso_corp, '/home/bmcfee/working/chords/model_deep_tv_struct_aug/predict_struct//')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [10]:
iso_chordino = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

A Jupyter Widget

In [11]:
iso_cr2psi = eval_model(iso_ids, iso_corp, '/home/bmcfee/working/chords//model_deep_pre_struct_aug/predict_struct//')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [18]:
iso_cr2npsi = eval_model(iso_ids, iso_corp, '/home/bmcfee/working/chords/model_noskip_pre_struct_aug/predict_struct/')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [25]:
iso_cr2nsi = eval_model(iso_ids, iso_corp, '/home/bmcfee/working/chords/model_noskip_struct_aug/predict_struct/')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [97]:
iso_cr2si = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions-inv/')

iso_cr2s = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions/')

iso_cr2 = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_aug/predictions/')

iso_cr1si = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions-inv/')

iso_cr1s = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions/')

iso_cr1 = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_aug/predictions/')

iso_chordino = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

iso_dnn = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_dnn/predictions/')

iso_khmm = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_khmm/predictions/')

A Jupyter Widget

/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [11]:
iso_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_chordino.json')

In [98]:
iso_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_chordino.json')
iso_dnn.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_dnn.json')
iso_khmm.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_khmm.json')

iso_cr1.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr1.json')
iso_cr1s.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr1s.json')
iso_cr1si.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr1si.json')

iso_cr2.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2.json')
iso_cr2s.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2s.json')
iso_cr2si.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2si.json')


In [26]:
iso_cr2nsi.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2nsi.json')

In [32]:
iso_cr2tvsi.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2tvsi.json')

In [19]:
iso_cr2npsi.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2npsi.json')

In [12]:
iso_cr2psi.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2psi.json')

In [13]:
# Billboard analysis
bb_ids = pd.read_csv('/home/bmcfee/git/ace-lessons/data/chord_dataset/billboard.csv', squeeze=True, header=None)
bb_corp = 'Billboard-Chords'

In [ ]:
bb_chordino = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

A Jupyter Widget

In [33]:
bb_cr2tvsi = eval_model(bb_ids, bb_corp, '/home/bmcfee/working/chords/model_deep_tv_struct_aug/predict_struct//')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [14]:
bb_cr2psi = eval_model(bb_ids, bb_corp, '/home/bmcfee/working/chords//model_deep_pre_struct_aug/predict_struct//')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [20]:
bb_cr2npsi = eval_model(bb_ids, bb_corp, '/home/bmcfee/working/chords/model_noskip_pre_struct_aug/predict_struct/')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [27]:
bb_cr2nsi = eval_model(bb_ids, bb_corp, '/home/bmcfee/working/chords/model_noskip_struct_aug/predict_struct/')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [101]:
bb_cr2si = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions-inv/')

bb_cr2s = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions/')

bb_cr2 = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_aug/predictions/')

bb_cr1si = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions-inv/')

bb_cr1s = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions/')

bb_cr1 = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_aug/predictions/')

bb_chordino = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

bb_dnn = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_dnn/predictions/')

bb_khmm = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_khmm/predictions/')

A Jupyter Widget

/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [ ]:
bb_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_chordino.json')

In [28]:
bb_cr2nsi.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2nsi.json')

In [34]:
bb_cr2tvsi.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2tvsi.json')

In [21]:
bb_cr2npsi.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2npsi.json')

In [15]:
bb_cr2psi.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2psi.json')

In [102]:
bb_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_chordino.json')
bb_dnn.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_dnn.json')
bb_khmm.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_khmm.json')

bb_cr1.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr1.json')
bb_cr1s.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr1s.json')
bb_cr1si.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr1si.json')

bb_cr2.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2.json')
bb_cr2s.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2s.json')
bb_cr2si.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2si.json')
